# Resolvendo o Jogo Mastermind com Programação Inteira

O Mastermind é um famoso jogo de tabuleiro, que consiste em adivinhar uma senha por tentativa e erro. A senha tem um certo comprimento $n$, e cada um dos $n$ caracteres é um pino de alguma cor, tomada dentre $d$ disponíveis - uma cor pode repetir-se na mesma senha ou não, dependendo da versão. A cada tentativa, o tabuleiro informa quantas posições foram preenchidas com a cor correta - essa quantidade é indicada por pontinhos pretos, mas a posição desses acertos, não. Além disso, é possível saber quantas das cores postas realmente ocorrem na senha, porém na posição errada - quantidade indicada por pontinhos brancos, mas que vamos desprezar na presente abordagem.

Há vários sítios na internet onde é possível **jogar gratuitamente**. Por exemplo, este:
https://www.matrixlab-examples.com/mastermind-game.html

O objetivo desta exposição é modelar o jogo como um problema de programação linear inteira, e desenvolver um código que permita acumular as informações dos erros até acertar a senha.

### Modelando o problema

O número de variáveis será $dn$. As variáveis $x_{ij}$, com $0 \le i \le d-1$ e $0 \le j \le n-1$, são as indicadoras de "a cor $i$ está correta na posição $j$", ou seja, $x_{ij}=1$, em caso afirmativo, e $x_{ij}=0$, caso contrário.

Naturalmente, cada coluna da matriz $X=(x_{ij}) \in M_{d \times n}(\mathbb{Z_2})$ tem soma $1$, pois cada posição $j$ admite uma única cor correta $i$.

Finalmente, cada tentativa $k$ terá um número de acertos $a_k$ com a string $s_k=s_{k0}s_{k1}\ldots s_{k(n-1)}$, que produzirá uma nova restrição sobre as variáveis: $$\displaystyle\sum_{j=0}^{n-1}x_{s_{kj}}=a_k$$

Então, eis as restrições a que o programa linear está sujeito:

---
* cada $x_{ij}$ é um inteiro igual a $0$ ou $1$;
* $\displaystyle\sum_{i=0}^{d-1}x_{ij}=1, \quad j=0,1,\ldots, n-1$;
* cada tentativa $k$ atualiza o programa com uma nova restrição $\displaystyle\sum_{j=0}^{n-1}x_{s_{kj}}=a_k$.


**Importante: não temos função objetivo!**

---

### Vamos utilizar a biblioteca OR-Tools

In [1]:
import ortools.linear_solver.pywraplp as otlp

### Criando um objeto Mastermind()

Este objeto, ao ser instanciado, cria o solver, recebe as dimensões $d$ e $n$ da matriz de variáveis, cria as variáveis indicadoras e adiciona as restrições das somas das colunas.

Ele conta com dois métodos.

Um método é o add_restr(), para adicionar novas restrições, a partir de uma senha tentada (pswd) e da sua quantidade de acertos (hit).

O outro é o new_guess(), que retorna uma sugestão de palpite: uma lista de $n$ inteiros de $0$ a $d-1$, possivelmente dentre várias existentes na região factível. Caso não exista solução viável - devido a alguma inconsistência lógica no jogo ou nas informações fornecidas -, o método simplesmente retorna False.

In [2]:
class Mastermind:

	def __init__(self, d, n):
		self.solver = otlp.Solver('euler185', otlp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

		self.digits = d
		self.length = n
		self.X = {}

		for i in range(d):
			self.X[i] = []
			for j in range(n):
				self.X[i].append(self.solver.IntVar(0,1,'x[{}][{}]'.format(i,j)))
	
		for j in range(n):
			self.solver.Add(self.solver.Sum(self.X[i][j] for i in range(d)) == 1)

	def add_restr(self, pswd, hit):
		self.solver.Add(self.solver.Sum(self.X[int(pswd[j])][j] for j in range(self.length)) == hit)

	def new_guess(self):
		try:
			self.solver.Solve()
			suggestion = ''
			for j in range(self.length):
				for i in range(self.digits):
					if self.X[i][j].solution_value() > 0.5:
						suggestion += ' ' + str(i)
						i += self.digits
			return suggestion
		except:
			return False

### Vencendo um jogo

Se estiver jogando alguma versão do jogo da senha e quiser conferir o objeto acima funcionando, use o pequeno programa a seguir. É um código rudimentar, bem pouco à prova de erros. Lembre-se de representar as cores por dígitos de $0$ a $d-1$, e de contar como acertos **somente os pontos pretos**. Você não precisa sempre seguir a senha sugerida (geralmente restam várias no conjunto viável), mas é indispensável fornecer sempre a quantidade correta de acertos.

Vale copiar de novo o link exibido no primeiro box:
https://www.matrixlab-examples.com/mastermind-game.html

In [ ]:
print('Quantas cores há? ')
d = int(input())
print('Qual é o comprimento da senha? ')
n = int(input())

master =  Mastermind(d,n)
terminated = False

print('Por favor, separe os dígitos da senha por espaços.')
while not terminated:
	print('Qual foi a última senha tentada? ')
	pswd = input().split()
	print('Quantos acertos ela teve? ')
	hit = int(input())

	if hit >= master.length:
		print('Parabéns! Você venceu!')
		break

	master.add_restr(pswd, hit)
	suggestion = master.new_guess()
	if suggestion:
		print('Tente a senha ', suggestion)
	else:
		print('Acabaram as sugestões!')
		terminated = True

Quantas cores há? 


# Resolvendo o problema 185 do Project Euler

Eis o enunciado:

O jogo Number Mind é uma variante do famoso Master Mind (também conhecido como Senha).

Em vez de pinos coloridos, se lhe pede que adivinhe uma sequência secreta de dígitos. Sobre cada palpite, só é informada a quantidade de dígitos corretos em posições corretas. Então, se a sequência era 1234 e o "chute" foi 2036, você seria informado de que acertou 1 dígito; contudo, não lhe seria dito que há um outro dígito, correto, porém na posição errada.

Por exemplo, dados os seguintes chutes para uma sequência secreta de cinco dígitos,

90342 ;2 corretos
70794 ;0 correto
39458 ;2 corretos
34109 ;1 correto
51545 ;2 corretos
12531 ;1 correto

A sequência correta 39542 é única.

Tome por base os seguintes palpites:

5616185650518293 ;2 corretos  
3847439647293047 ;1 correto  
5855462940810587 ;3 corretos  
9742855507068353 ;3 corretos  
4296849643607543 ;3 corretos  
3174248439465858 ;1 correto  
4513559094146117 ;2 corretos  
7890971548908067 ;3 corretos  
8157356344118483 ;1 correto  
2615250744386899 ;2 corretos  
8690095851526254 ;3 corretos  
6375711915077050 ;1 correto  
6913859173121360 ;1 correto  
6442889055042768 ;2 corretos  
2321386104303845 ;0 correto  
2326509471271448 ;2 corretos  
5251583379644322 ;2 corretos  
1748270476758276 ;3 corretos  
4895722652190306 ;1 correto  
3041631117224635 ;3 corretos  
1841236454324589 ;3 corretos  
2659862637316867 ;2 corretos  

Ache a única sequência secreta de 16 dígitos.

In [ ]:
import pandas as pd

guesses = pd.read_csv('project-euler-185.csv')
G = guesses.iloc[:,:-1].values
h = guesses.iloc[:,-1].values

n = len(G[0])
master = Mastermind(10,n)

for k in range(len(h)):
        master.add_restr(G[k], h[k])
        
print('A senha é ', master.new_guess())